# Denison DA210/CS181 Homework 2.d - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

In [1]:
import os
import os.path
import pandas as pd

datadir = "publicdata"

---

## Part A: Aggregating a `Series`/`DataFrame`

**Q1:** Read CSV file `topnames.csv` in `datadir` into a data frame named `topnames0`, with no index.  Using individual operations on the `count` column, find the mean, the median, and the max count, assigning to `mean_counts`, `median_counts`, and `max_counts`.

In [30]:
file = pd.read_csv(os.path.join(datadir, "topnames.csv"))
topnames0 = pd.DataFrame(file)
#mean
mean_counts = topnames0["count"].mean()
#median 
median_counts = topnames0["count"].median()
#max
max_counts = topnames0["count"].max()



In [31]:
# Printing cell
print("Mean:", mean_counts,
      "Median:", median_counts,
      "Max:", max_counts)

Mean: 44985.24460431655 Median: 52556.0 Max: 99689


In [32]:
# Testing Cell
assert abs(mean_counts - 44985.2446) < 0.001
assert median_counts == 52556.0
assert max_counts == 99689

**Q2:** Using the `agg` method on the `Series` of the column vector of counts, perform the same calculation of mean, median, and max in a single step, and assign to `agg_values`.  Note that this invocation may not have an exact correspondent in the book, so you may have to look up [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.agg.html) for using `agg` on a `Series`.

In [84]:

agg_values = topnames0["count"].agg(["mean","median","max"])

# Display the aggregated values
agg_values

type(agg_values)

pandas.core.series.Series

In [47]:
# Testing Cell
assert abs(agg_values.loc["mean"] - 44985.2446) < 0.001
assert agg_values.loc["median"] == 52556.0
assert agg_values.loc["max"] == 99689.0

**Q3:** What is the data type of `agg_values`?  (I.e., what is the result of using `agg` on a `Series`?)

agg_values is a Series

**Q4:** Create a subset of `topnames0` restricted to `Female` entries between `1960` and `1969`, inclusive.  Assign this to `female_subset`.  Then use the `agg` function, in one step, to determine the mean and median count and the number of unique names, assigning to `female_aggvalues`.

In [109]:
female_subset = topnames0[(topnames0["sex"] == "Female") & (topnames0["year"] >= 1960) & (topnames0["year"] <= 1969)]


female_aggvalues = female_subset.agg({"count": ["mean","median"], "name":["nunique"]})

# Display the aggregated values
female_aggvalues

pandas.core.frame.DataFrame

In [108]:
# Testing Cell
assert female_aggvalues.shape == (3,2)
assert female_aggvalues.loc["mean", "count"] == 51972.7
assert female_aggvalues.loc["median", "count"] == 51955.0
assert female_aggvalues.loc["nunique", "name"] == 2.0

**Q5:** What is the data type of `female_aggvalues`?  (I.e., what is the result of using `agg` on a subset of a `DataFrame`?)

female_aggvalues is a DataFrame 

**Q6:** The constraints for selecting the rows from the last problem are based on `sex` and `year`.  We often use these independent variables to set an index for a data set.  Then, when we want to filter rows, our operations that use row label/`index` values for filtering are different.

Start by creating dataframe `topnames` with its index drawn from the columns `year` and `sex`.  Then, with a goal of the same use of the `agg` function from **Q4**, use `xs` to take a cross section of `topnames` to get the Female entries and then use `loc` to get a data frame, `female_xs_subset`, for the year range from 1960 to 1969, inclusive.  Finally, use `agg` on this data frame to, in one step, determine the mean and median count and the number of unique names, assigning to `female_xs_aggvalues`.

In [137]:
topnames = topnames0.set_index(["sex","year"], inplace=False)

female_xs_subset = topnames.xs(key = "Female", level = "sex")


female_xs_subset = female_xs_subset.loc[1960:1969, :]

female_xs_aggvalues = female_xs_subset.agg({"count": ["mean","median"], "name":["nunique"]})
# Display the aggregated values

female_xs_aggvalues


,count,name
mean,51972.7,NaN
median,51955.0,NaN
nunique,NaN,2.0


In [138]:
# Testing Cell
assert female_xs_aggvalues.shape == (3,2)
assert female_xs_aggvalues.loc["mean", "count"] == 51972.7
assert female_xs_aggvalues.loc["median", "count"] == 51955.0
assert female_xs_aggvalues.loc["nunique", "name"] == 2.0

---

## Part B: GroupBy

**Q7:** Read CSV file `indicators2016.csv` in `datadir` into a data frame named `indicators0`, with no index. Write code to add a new column `popSize` to `indicators0` which takes value `"high"` if `pop > 300`, `"low"` if `pop < 50`, and `"medium"` otherwise.

In [144]:
indicators0 = pd.DataFrame(pd.read_csv(os.path.join(datadir, "indicators2016.csv")))
def check(x): 
    if x >300:
        return "high"
    elif x <50: 
        return "low"
    else:
        return "medium"

size = indicators0["pop"].apply(check)

indicators0["popSize"] = size

indicators0 
# Display part of the DataFrame
#indicators0.head()

,code,country,pop,gdp,life,cell,popSize
0,ABW,Aruba,0.10,NaN,75.87,NaN,low
1,AFG,Afghanistan,34.66,19.47,63.67,21.60,low
2,AGO,Angola,28.81,95.34,61.55,13.00,low
3,ALB,Albania,2.88,11.86,78.34,3.37,low
4,AND,Andorra,0.08,2.86,NaN,0.07,low
...,...,...,...,...,...,...,...
215,XKX,Kosovo,1.82,6.65,71.65,NaN,low
216,YEM,"Yemen, Rep.",27.58,27.32,64.95,16.43,low
217,ZAF,South Africa,56.02,295.46,62.77,82.41,medium
218,ZMB,Zambia,16.59,21.06,61.87,12.02,low


In [145]:
# Testing Cell
assert indicators0["popSize"].nunique() == 3
assert indicators0["popSize"].value_counts()["low"] == 189
assert indicators0["popSize"].value_counts()["medium"] == 25
assert indicators0["popSize"].value_counts()["high"] == 6
assert indicators0.loc[205, "popSize"] == "high"
assert indicators0.loc[17, "popSize"] == "medium"
assert indicators0.loc[4, "popSize"] == "low"

**Q8:** Building on the question above, use `groupby` to partition `indicators0` by this new column `popSize`, assigning to variable `groupby_pop`.

In [158]:
groupby_pop = indicators0.groupby("popSize")

# Print the number of groups
print(len(groupby_pop))


3


In [147]:
# Testing Cell
assert len(groupby_pop) == 3

**Q9:** What is the data type of `groupby_pop`?  (I.e., what is the result of using `groupby` on a `DataFrame`?)

When I use type(groupby_pop) it says that groupby_pop is a generic dataframe groupby(?), so I guess it is a dataframe that has an indicator column as "high", "low", and "medium". 

**Q10:** Building on the question above, aggregate this groupby partitioning, determining the **number of non-missing elements** for the columns `code`, `pop`, `gdp`, `life`, and `cell` by partition.  Assign the result to `partition_counts`.

Hint: You can make this general by using a list-comprehension-type expression to create a dictionary.  (This is not necessary, but cool to use.)

In [160]:
aggs = {"code":"count", "pop":"count","gdp":"count", "life":"count", "cell": "count"}

partition_counts = groupby_pop.agg(aggs)

# Display the grouped counts
partition_counts

,code,pop,gdp,life,cell
popSize,,,,,
high,6,6,6,6,6
low,189,189,164,171,172
medium,25,24,24,25,25


In [161]:
# Testing Cell
assert partition_counts.shape == (3,5)
assert partition_counts.loc["low", "code"] == 189
assert partition_counts.loc["medium", "gdp"] == 24
assert partition_counts.loc["high", "cell"] == 6

**Q11:** In similar fashion, determine the mean, by partition, of `pop`, `gdp`, and `cell`, and the max of `life`, assigning to `partition_aggvalues`.  Use the `round()` method of DataFrames to round the numeric values to 2 decimal places.

In [167]:
aggs = {"pop":"mean","gdp":"mean", "life":"max", "cell": "mean"}

partition_aggvalues = groupby_pop.agg(aggs).round(2)

# Display the grouped aggregated values
partition_aggvalues

,pop,gdp,life,cell
popSize,,,,
high,2100.92,18715.31,78.69,2040.46
low,9.44,101.67,84.23,10.26
medium,108.52,1076.00,83.98,113.07


In [168]:
# Testing Cell
assert partition_aggvalues.shape == (3,4)
assert partition_aggvalues.loc["low", "gdp"] == 101.67
assert partition_aggvalues.loc["medium", "life"] == 83.98
assert partition_aggvalues.loc["high", "pop"] == 2100.92

**Q12:** In similar fashion, determine the mean, min, and max of `gdp` and `life`, assigning to `partition_aggvalues2` and again rounding to two decimal places.

In [169]:
aggs = {"gdp":["mean","min", "max"], "life":["mean","min", "max"]}

partition_aggvalues2 = groupby_pop.agg(aggs).round(2)

# Display the grouped aggregated values

# Display the grouped aggregated values
partition_aggvalues2

gdp                      life              
             mean      min       max   mean    min    max
popSize                                                  
high     18715.31  1440.24  75871.74  72.87  68.56  78.69
low        101.67     0.03   1535.77  72.15  51.84  84.23
medium    1076.00    35.38   4949.27  72.29  53.43  83.98

In [170]:
# Testing Cell
assert partition_aggvalues2.shape == (3,6)
assert partition_aggvalues2.loc["low", ("gdp", "mean")] == 101.67
assert partition_aggvalues2.loc["medium", ("life", "max")] == 83.98
assert partition_aggvalues2.loc["high", ("gdp", "min")] == 1440.24

**Q13:** Explain how the columns of `partition_aggvalues` and `partition_aggvalues2` differ, and **why**.

For partition_aggvalues, we are finding the mean for 3 variables and the max for 1 varible; thus, each column represents an operational output for the according variable. 
Meanwhile, partition_aggvalues2 represents 3 different operations (mean, min, max) for 2 different variables. Hence, we can see gdp and life (the two interested variables) as the headers for the table, followed by 3 columns which are the operations we wanted to find the values of. 

---

---
## Part C

**Q14:** How much time (in minutes/hours) did you spend on this homework assignment?

1.5 hour 

**Q15:** Who was your partner for this assignment?  If you worked alone, say so instead.

Alone